In [2]:
from pyspark.sql import SparkSession

In [5]:
spark = (
    SparkSession.
    builder.
    appName("3_handling_missing_vaules").
    getOrCreate()
)
spark

24/03/14 06:35:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Load df

In [9]:
!wget https://raw.githubusercontent.com/krishnaik06/Pyspark-With-Python/main/test2.csv

--2024-03-14 06:58:37--  https://raw.githubusercontent.com/krishnaik06/Pyspark-With-Python/main/test2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180 [text/plain]
Saving to: ‘test2.csv.1’

test2.csv.1         100%[===================>]     180  --.-KB/s    in 0s      

2024-03-14 06:58:38 (7.37 MB/s) - ‘test2.csv.1’ saved [180/180]



In [10]:
df = spark.read.csv("./test2.csv", inferSchema=True, header=True)

In [12]:
df.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



In [15]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



## Drop null values

In [18]:
df.na.drop().show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [24]:
## any == how
df.na.drop(how='any').show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [27]:
# if thresh == 3, minimum 3 non null value must present
df.na.drop(how='any', thresh=3).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     NULL| 34|        10| 38000|
+---------+---+----------+------+



In [29]:
df.na.drop(how='any', subset=['Name', 'Salary']).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
+---------+----+----------+------+



## Filling missing value

In [45]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [43]:
df.na.fill("NOT FOUND").show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|NOT FOUND|  34|        10| 38000|
|NOT FOUND|  36|      NULL|  NULL|
+---------+----+----------+------+



In [44]:
df.na.fill(0, subset=['Experience']).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|         0| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|         0|  NULL|
+---------+----+----------+------+



['Name', 'age', 'Experience', 'Salary']

## Fill na values with imputer

In [47]:
from pyspark.ml.feature import Imputer

In [61]:
cols = ["age", "Experience", "Salary"]
out_cols = [f"{i}_imputted" for i in cols]

In [67]:
imputer = Imputer()
imputer.setInputCols(cols)
imputer.setOutputCols(out_cols)
imputer.setStrategy('mean')

Imputer_6cc436747b08

In [68]:
# imputer = Imputer(
#     inputCols=cols, outputCols=out_cols
# ).setStrategy('mean')

In [69]:
imputer.fit(df).transform(df).show()

+---------+----+----------+------+------------+-------------------+---------------+
|     Name| age|Experience|Salary|age_imputted|Experience_imputted|Salary_imputted|
+---------+----+----------+------+------------+-------------------+---------------+
|    Krish|  31|        10| 30000|          31|                 10|          30000|
|Sudhanshu|  30|         8| 25000|          30|                  8|          25000|
|    Sunny|  29|         4| 20000|          29|                  4|          20000|
|     Paul|  24|         3| 20000|          24|                  3|          20000|
|   Harsha|  21|         1| 15000|          21|                  1|          15000|
|  Shubham|  23|         2| 18000|          23|                  2|          18000|
|   Mahesh|NULL|      NULL| 40000|          28|                  5|          40000|
|     NULL|  34|        10| 38000|          34|                 10|          38000|
|     NULL|  36|      NULL|  NULL|          36|                  5|         